1. Import libraries & CitiBike data.
2. Get weather data from NOAA.
3. Build the weather dataframe.
4. Merge the CitiBike and LaGuardia weather data.

In [6]:
# Import libraries
from pathlib import Path
import os
import pandas as pd

# Set path
PROJECT_ROOT = Path.cwd()
PROJECT_ROOT

# Path to the folder that contains all month subfolders
DATA_ROOT = PROJECT_ROOT / "data" / "raw" / "2022-citibike-tripdata"

DATA_ROOT

PosixPath('/Users/samantha.lisik/Documents/citibike/data/raw/2022-citibike-tripdata')

In [7]:
# Verify folder contents
list(DATA_ROOT.iterdir())

[PosixPath('/Users/samantha.lisik/Documents/citibike/data/raw/2022-citibike-tripdata/202211-citibike-tripdata'),
 PosixPath('/Users/samantha.lisik/Documents/citibike/data/raw/2022-citibike-tripdata/202208-citibike-tripdata'),
 PosixPath('/Users/samantha.lisik/Documents/citibike/data/raw/2022-citibike-tripdata/.DS_Store'),
 PosixPath('/Users/samantha.lisik/Documents/citibike/data/raw/2022-citibike-tripdata/202210-citibike-tripdata'),
 PosixPath('/Users/samantha.lisik/Documents/citibike/data/raw/2022-citibike-tripdata/202209-citibike-tripdata'),
 PosixPath('/Users/samantha.lisik/Documents/citibike/data/raw/2022-citibike-tripdata/202212-citibike-tripdata'),
 PosixPath('/Users/samantha.lisik/Documents/citibike/data/raw/2022-citibike-tripdata/202202-citibike-tripdata'),
 PosixPath('/Users/samantha.lisik/Documents/citibike/data/raw/2022-citibike-tripdata/202207-citibike-tripdata'),
 PosixPath('/Users/samantha.lisik/Documents/citibike/data/raw/2022-citibike-tripdata/202203-citibike-tripdata')

In [8]:
# Recursively find all CSV files inside month folders
filepaths = list(DATA_ROOT.rglob("*.csv"))

len(filepaths), filepaths[:5]

(36,
 [PosixPath('/Users/samantha.lisik/Documents/citibike/data/raw/2022-citibike-tripdata/202211-citibike-tripdata/202211-citibike-tripdata_3.csv'),
  PosixPath('/Users/samantha.lisik/Documents/citibike/data/raw/2022-citibike-tripdata/202211-citibike-tripdata/202211-citibike-tripdata_2.csv'),
  PosixPath('/Users/samantha.lisik/Documents/citibike/data/raw/2022-citibike-tripdata/202211-citibike-tripdata/202211-citibike-tripdata_1.csv'),
  PosixPath('/Users/samantha.lisik/Documents/citibike/data/raw/2022-citibike-tripdata/202208-citibike-tripdata/202208-citibike-tripdata_3.csv'),
  PosixPath('/Users/samantha.lisik/Documents/citibike/data/raw/2022-citibike-tripdata/202208-citibike-tripdata/202208-citibike-tripdata_2.csv')])

In [12]:
# Merge all 36 CSV files into a single dataframe
%time

citibike_2022 = pd.concat(
    (pd.read_csv(fp) for fp in filepaths),   # generator expression: efficient & memory-safe
    ignore_index=True
)

citibike_2022.shape

CPU times: user 2 μs, sys: 0 ns, total: 2 μs
Wall time: 5.01 μs


/var/folders/s0/tyllrxjd18l9z7g_0x8hn4x80000gp/T/ipykernel_24519/2276601960.py:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  (pd.read_csv(fp) for fp in filepaths),   # generator expression: efficient & memory-safe
/var/folders/s0/tyllrxjd18l9z7g_0x8hn4x80000gp/T/ipykernel_24519/2276601960.py:5: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  (pd.read_csv(fp) for fp in filepaths),   # generator expression: efficient & memory-safe
/var/folders/s0/tyllrxjd18l9z7g_0x8hn4x80000gp/T/ipykernel_24519/2276601960.py:5: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  (pd.read_csv(fp) for fp in filepaths),   # generator expression: efficient & memory-safe
/var/folders/s0/tyllrxjd18l9z7g_0x8hn4x80000gp/T/ipykernel_24519/2276601960.py:5: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=

(29838806, 13)

In [13]:
# Check the dataframe
citibike_2022.shape

(29838806, 13)

In [14]:
# Create the date column
citibike_2022["started_at"] = pd.to_datetime(citibike_2022["started_at"])
citibike_2022["date"] = citibike_2022["started_at"].dt.date

citibike_2022[["started_at", "date"]].head()

,started_at,date
0,2022-11-02 05:54:11.481,2022-11-02
1,2022-11-02 18:20:15.611,2022-11-02
2,2022-11-04 18:39:39.873,2022-11-04
3,2022-11-09 18:02:29.616,2022-11-09
4,2022-11-12 10:23:11.805,2022-11-12


# How the Citi Bike data was imported and joined

I used Path.rglob("*.csv") to automatically find every Citi Bike CSV file inside the 2022 data folder and all its monthly subfolders. This creates a list of file paths without needing to reference each file manually.

To combine the files, I used a generator expression inside pd.concat, which reads each CSV one at a time and efficiently stacks them into a single dataframe.

This approach is concise, memory-efficient, and ideal for joining many similarly structured datasets.

2. Get weather data from NOAA. 

In [15]:
# Load NOAA token
from dotenv import load_dotenv
import os

load_dotenv()

NOAA_TOKEN = os.getenv("NOAA_TOKEN")
NOAA_TOKEN

'iXBuiPQVGdUXnvmuvxjffFVxZPHXKgNE'

In [16]:
# Set up the NOAA API request (LaGuardia Airport, 2022)
import requests
import json
from datetime import datetime

BASE_URL = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"

params = {
    "datasetid": "GHCND",                 # daily climate data
    "datatypeid": "TAVG",                 # average temperature
    "stationid": "GHCND:USW00014732",     # LaGuardia Airport
    "startdate": "2022-01-01",
    "enddate": "2022-12-31",
    "limit": 1000,                        # enough for a full year
}

headers = {"token": NOAA_TOKEN}

In [17]:
# Make the request and inspect
r = requests.get(BASE_URL, params=params, headers=headers)
r.status_code

200

In [18]:
d = json.loads(r.text)
d.keys()

dict_keys(['metadata', 'results'])

In [19]:
# Filter the weather data for the daily average temperature (TAVG)
avg_temps = [item for item in d["results"] if item["datatype"] == "TAVG"]

len(avg_temps)

365

In [20]:
# Extract dates and raw temperature values
date_strings = [item["date"] for item in avg_temps]
temp_values_raw = [item["value"] for item in avg_temps]

In [21]:
# Build a clean weather dataframe
df_weather = pd.DataFrame()

# Convert string timestamps ("2022-01-01T00:00:00") --> real date objects
df_weather["date"] = [
    datetime.strptime(ds, "%Y-%m-%dT%H:%M:%S").date()
    for ds in date_strings
]

# Convert tenths-of-degree values to actual Celsius
df_weather["avgTemp_C"] = [v / 10.0 for v in temp_values_raw]

df_weather.head()

,date,avgTemp_C
0,2022-01-01,11.6
1,2022-01-02,11.4
2,2022-01-03,1.4
3,2022-01-04,-2.7
4,2022-01-05,3.2


# How I built the weather dataframe

The NOAA API returned a JSON object containing many different weather measurements. I filtered the results to keep only "TAVG" (average daily temperature). I then extracted the dates and the raw temperature values.

Because NOAA stores temperatures in tenths of degrees Celsius, I divided each value by 10 to convert them to actual Celsius. I also converted the date strings (e.g., "2022-01-01T00:00:00") into Python date objects. Finally, I combined these cleaned values into a clean dataframe with two columns: date and avgTemp_C.

4. Merge the CitiBike and LaGuardia weather data.

In [22]:
# Save the weather data as a CSV file in the "external" folder
external_dir = PROJECT_ROOT / "data" / "external"

weather_path = external_dir / "laguardia_2022_daily_tavg.csv"
df_weather.to_csv(weather_path, index=False)

weather_path

PosixPath('/Users/samantha.lisik/Documents/citibike/data/external/laguardia_2022_daily_tavg.csv')

In [23]:
# Ensure both dataframes have matching date types
citibike_2022["date"] = pd.to_datetime(citibike_2022["date"]).dt.date
df_weather["date"] = pd.to_datetime(df_weather["date"]).dt.date

In [24]:
# Merge on "date"
%time

df_merged = citibike_2022.merge(
    df_weather,
    how="left",     # keep all Citi Bike trips and attach weather
    on="date",
    indicator=True
)

df_merged["_merge"].value_counts(dropna=False)

CPU times: user 4 μs, sys: 0 ns, total: 4 μs
Wall time: 5.72 μs


_merge
both          29838166
left_only          640
right_only           0
Name: count, dtype: int64

In [25]:
# Inspect the merged data
df_merged[["started_at", "date", "avgTemp_C"]].head()

,started_at,date,avgTemp_C
0,2022-11-02 05:54:11.481,2022-11-02,17.6
1,2022-11-02 18:20:15.611,2022-11-02,17.6
2,2022-11-04 18:39:39.873,2022-11-04,15.2
3,2022-11-09 18:02:29.616,2022-11-09,8.7
4,2022-11-12 10:23:11.805,2022-11-12,19.7


In [26]:
# Check which rows are missing weather data
df_merged[df_merged["_merge"] == "left_only"][["date", "avgTemp_C"]].head()
df_merged["avgTemp_C"].isna().sum()

np.int64(640)

- 29,838,166 trip records successfully matched a weather date
- 640 trip records do not have corresponding temperature data
- This is normal with NOAA data. A few reasons this happens:

NOAA sometimes lacks TAVG readings for particular days.

Some dates may be missing from the station record.

Some timestamps in Citi Bike data may be outside the 2022-01-01 → 2022-12-31 request range.

Weather data might not exist for certain maintenance or outage days.

In [27]:
# Export the merged dataset to CSV
processed_dir = PROJECT_ROOT / "data" / "processed"
merged_path = processed_dir / "citibike_2022_with_weather.csv"

df_merged.to_csv(merged_path, index=False)

merged_path

PosixPath('/Users/samantha.lisik/Documents/citibike/data/processed/citibike_2022_with_weather.csv')